In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from basemodels.models import FikstureModelData

In [3]:
#verileri oku
,
import pandas as pd

data = FikstureModelData.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df

,id,data_id,tarih,data,count,isprogress,created_at,updated_at
0,1,20241021,2024-10-21,"[{'tournament': {'name': 'Premier League', 'sl...",333,False,2024-10-21 11:18:48.967709+00:00,2024-10-21 11:18:48.967709+00:00
1,6,20241020,2024-10-20,"[{'tournament': {'name': 'Premier League', 'sl...",744,False,2024-10-21 12:03:20.963828+00:00,2024-10-21 12:03:20.963828+00:00
2,13,20241019,2024-10-19,"[{'tournament': {'name': 'Premier League', 'sl...",826,False,2024-10-21 12:24:06.417397+00:00,2024-10-21 12:24:06.417397+00:00
3,16,20241018,2024-10-18,"[{'tournament': {'name': 'Premier League', 'sl...",411,False,2024-10-21 13:04:53.455994+00:00,2024-10-21 13:04:53.455994+00:00
4,20,20241017,2024-10-17,[{'tournament': {'name': 'Brasileirão Série A'...,83,False,2024-10-21 13:10:11.208836+00:00,2024-10-21 13:10:11.208836+00:00
5,21,20241016,2024-10-16,"[{'tournament': {'name': 'UEFA Nations League,...",120,False,2024-10-21 13:14:11.457338+00:00,2024-10-21 13:14:11.457338+00:00
6,22,20241001,2024-10-01,[{'tournament': {'name': 'UEFA Champions Leagu...,176,False,2024-10-21 13:27:48.695696+00:00,2024-10-21 13:27:48.695696+00:00
7,23,20241002,2024-10-02,[{'tournament': {'name': 'UEFA Champions Leagu...,206,False,2024-10-21 13:27:53.223685+00:00,2024-10-21 13:27:53.223685+00:00
8,24,20241003,2024-10-03,[{'tournament': {'name': 'UEFA Champions Leagu...,203,False,2024-10-21 13:27:56.535289+00:00,2024-10-21 13:27:56.535289+00:00
9,25,20241004,2024-10-04,"[{'tournament': {'name': 'Premier League', 'sl...",422,False,2024-10-21 13:28:00.910519+00:00,2024-10-21 13:28:00.910519+00:00


In [4]:
from basemodels.models import Tournament,Season

# Modelleri import edin
import pandas as pd

class SofascoreAPI:
    def __init__(self, date):
        self.date = date
        
    def fetch_data(self):
        
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]

        return veri
        

    def tournament_data(self):

            verim = self.fetch_data()
            """Eksik sütunları kontrol edip, varsa 0 ile doldurma ve veriyi sonlandırma."""
            if verim is None:
                raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
            
            # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
            verim = verim.drop_duplicates("tournament_uniqueTournament_id")
            
            # İlgili sütunlar
            cols = [
                "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
                "tournament_category_slug", "tournament_uniqueTournament_category_id", 
                "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
                "tournament_uniqueTournament_hasPerformanceGraphFeature"
            ]
            
            # Eksik sütunları 0 ile doldurma
            for col in cols:
                if col not in verim.columns:
                    verim.loc[:, col] = 0
    
            # Son olarak sadece belirtilen sütunlar kalsın
            verim = verim[cols]

            for _, row in verim.iterrows():
                tournament_id = row['tournament_uniqueTournament_id']
                
                # NaN değerlerini kontrol edin ve NaN olan satırları atlayın
                if pd.isna(tournament_id):
                    print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                    continue  # NaN olan satırı atla
        
                # Modelde bu id ile kayıt olup olmadığını kontrol et
                if not Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).exists():
                    # Kayıt yoksa yeni kayıt oluştur
                    Tournament.objects.create(
                        tournament_uniqueTournament_id=row['tournament_uniqueTournament_id'],
                        tournament_name=row['tournament_name'],
                        tournament_slug=row['tournament_slug'],
                        tournament_category_name=row['tournament_category_name'],
                        tournament_category_slug=row['tournament_category_slug'],
                        tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                        tournament_uniqueTournament_hasEventPlayerStatistics=row['tournament_uniqueTournament_hasEventPlayerStatistics'],
                        tournament_id=row['tournament_id'],
                        tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                        tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                    )
                    print(f"Tournament ID {tournament_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        """Season verileri için eksik sütunları kontrol edip, 0 ile doldurma işlemi."""
        verim = self.fetch_data()
        
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı season_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("season_id")
        
        # Sezonla ilgili sütunlar
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        # Eksik sütunları 0 ile doldurma
        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        # Son olarak sadece season ile ilgili sütunları döndür
        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')  # İlgili turnuvayı al
    
            # NaN değerlerini kontrol edin ve geçersiz veriyi atlayın
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            # Modelde bu id ile kayıt olup olmadığını kontrol et
            if not Season.objects.filter(season_id=season_id).exists():
                # Turnuva olup olmadığını kontrol et ve turnuva ile sezonu ilişkilendir
                tournament_instance = Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).first()
                
                if tournament_instance:
                    # Kayıt yoksa yeni sezon kaydı oluştur
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Turnuva ile ilişkilendir
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")



tarih = "2023-10-20"

api = SofascoreAPI(tarih)  
api.tournament_data() 
api.season_data()

Tournament ID 17 zaten mevcut, eklenmedi.
Tournament ID 8 zaten mevcut, eklenmedi.
Tournament ID 23 zaten mevcut, eklenmedi.
Tournament ID 35 zaten mevcut, eklenmedi.
Tournament ID 34 zaten mevcut, eklenmedi.
Tournament ID 37 zaten mevcut, eklenmedi.
Tournament ID 325 zaten mevcut, eklenmedi.
Tournament ID 52 zaten mevcut, eklenmedi.
Tournament ID 38 zaten mevcut, eklenmedi.
Tournament ID 185 zaten mevcut, eklenmedi.
Tournament ID 2132 zaten mevcut, eklenmedi.
Tournament ID 155 zaten mevcut, eklenmedi.
Tournament ID 11621 zaten mevcut, eklenmedi.
Tournament ID 170 zaten mevcut, eklenmedi.
Tournament ID 210 zaten mevcut, eklenmedi.
Tournament ID 18 zaten mevcut, eklenmedi.
Tournament ID 54 zaten mevcut, eklenmedi.
Tournament ID 53 zaten mevcut, eklenmedi.
Tournament ID 182 zaten mevcut, eklenmedi.
Tournament ID 390 zaten mevcut, eklenmedi.
Tournament ID 131 zaten mevcut, eklenmedi.
Tournament ID 98 zaten mevcut, eklenmedi.
Tournament ID 45 zaten mevcut, eklenmedi.
Tournament ID 266 zate